In [2]:
import sqlalchemy
import requests
import json

response_API = requests.get('https://restcountries.com/v3.1/all')
#print(response_API.status_code)
data = json.loads(response_API.text)

In [16]:
json_formatted_str = json.dumps(data[0], indent=2)

print(json_formatted_str)

{
  "name": {
    "common": "Jordan",
    "official": "Hashemite Kingdom of Jordan",
    "nativeName": {
      "ara": {
        "official": "\u0627\u0644\u0645\u0645\u0644\u0643\u0629 \u0627\u0644\u0623\u0631\u062f\u0646\u064a\u0629 \u0627\u0644\u0647\u0627\u0634\u0645\u064a\u0629",
        "common": "\u0627\u0644\u0623\u0631\u062f\u0646"
      }
    }
  },
  "tld": [
    ".jo",
    "\u0627\u0644\u0627\u0631\u062f\u0646."
  ],
  "cca2": "JO",
  "ccn3": "400",
  "cca3": "JOR",
  "cioc": "JOR",
  "independent": true,
  "status": "officially-assigned",
  "unMember": true,
  "currencies": {
    "JOD": {
      "name": "Jordanian dinar",
      "symbol": "\u062f.\u0627"
    }
  },
  "idd": {
    "root": "+9",
    "suffixes": [
      "62"
    ]
  },
  "capital": [
    "Amman"
  ],
  "altSpellings": [
    "JO",
    "Hashemite Kingdom of Jordan",
    "al-Mamlakah al-Urdun\u012byah al-H\u0101shim\u012byah"
  ],
  "region": "Asia",
  "subregion": "Western Asia",
  "languages": {
    "ara": "Arabic

In [19]:
data[0].keys()

dict_keys(['name', 'tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status', 'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region', 'subregion', 'languages', 'translations', 'latlng', 'landlocked', 'borders', 'area', 'demonyms', 'flag', 'maps', 'population', 'gini', 'fifa', 'car', 'timezones', 'continents', 'flags', 'coatOfArms', 'startOfWeek', 'capitalInfo', 'postalCode'])

In [3]:
import pandas

def return_if_exists(country,field):
    try:
        return country[field]
    except:
        return None

myList = []
for country in data:
    myList.append(
        {
            'nombre' : country['name']['common'],
            'capital' : country['capital'], return_if_exists(country,'capital')
            'poblacion' : lambda x: country['population'],
            'bandera' : country['flag']
        }
    )
df = pd.DataFrame(myList)

data[0]['capital']

KeyError: 'capital'

In [4]:
data[5]['capital']

['Hamilton']

In [5]:
paises_sin_currency = 0
paises_multiples_currency =0
for i in range(len(data)):
    try:
        if len(data[i]['continents']) != 1:   
            paises_multiples_currency +=1
    except:
        paises_sin_currency +=1

print(f'paises sin currency: {paises_sin_currency}')
print(f'paises con multiples currencies: {paises_multiples_currency}')

paises sin currency: 0
paises con multiples currencies: 0


In [71]:
data[9]['flag']

'🇲🇱'

In [82]:
for i in range(len(data)):
    if len(data[i]['flag']) != 2:
        print(i)


In [84]:
data[150]['capital']

['Moscow']

In [75]:
data[150]

{'name': {'common': 'Russia',
  'official': 'Russian Federation',
  'nativeName': {'rus': {'official': 'Российская Федерация',
    'common': 'Россия'}}},
 'tld': ['.ru', '.su', '.рф'],
 'cca2': 'RU',
 'ccn3': '643',
 'cca3': 'RUS',
 'cioc': 'RUS',
 'independent': True,
 'status': 'officially-assigned',
 'unMember': True,
 'currencies': {'RUB': {'name': 'Russian ruble', 'symbol': '₽'}},
 'idd': {'root': '+7', 'suffixes': ['3', '4', '5', '8', '9']},
 'capital': ['Moscow'],
 'altSpellings': ['RU', 'Russian Federation', 'Российская Федерация'],
 'region': 'Europe',
 'subregion': 'Eastern Europe',
 'languages': {'rus': 'Russian'},
 'translations': {'ara': {'official': 'روسيا الاتحادية', 'common': 'روسيا'},
  'bre': {'official': 'Kevread Rusia', 'common': 'Rusia'},
  'ces': {'official': 'Ruská federace', 'common': 'Rusko'},
  'cym': {'official': 'Russian Federation', 'common': 'Russia'},
  'deu': {'official': 'Russische Föderation', 'common': 'Russland'},
  'est': {'official': 'Venemaa Föder